### Step 1: Reading the data

In [0]:
userJsonPath = "/FileStore/tables/Project_Data_Extract/users.json"
users_DF = (spark.read.option("inferSchema", True).json(userJsonPath))
usersDF.display()

In [0]:
usersDF.printSchema()

### Step 2: Data Preparation

### Step 2.1: Filter out non f&b business

In [0]:
users_pdf=userDF.toPandas()

In [0]:
bizDF = (spark.read
  .option("inferSchema", True)
  .json("/FileStore/tables/Project_Data_Extract/dataset_business_cleaned.json"))

biz_pdf=bizDF.toPandas()
biz_id_list=biz_pdf['business_id'].unique()
biz_id_list

In [0]:
users_pdf.shape

In [0]:
users_pdf=users_pdf[users_pdf['business_id'].isin(biz_id_list)]

In [0]:
users_pdf.shape

### Step 2.2: Check for missing values

In [0]:
users_pdf.isnull().sum()

### Step 2.3: Add elite count column

In [0]:
import pandas as pd
elite_count = [0 for _ in range(user_pdf.shape[0])]
for i in range(user_pdf.shape[0]):
    elite_count[i] = len(user_pdf.loc[i, "elite"].split(","))
    
elite_count = pd.DataFrame(elite_count)
# print(elite_count)
user_pdf['elite_count'] = elite_count  # Add column to user_pdf
user_pdf = user_pdf.drop(["elite"], axis=1) # Drop elite
user_pdf.head()

### Step 2.4: Add year column for yelping_since

In [0]:
user_pdf['yelping_since'] = pd.to_datetime(user_pdf['yelping_since'])
 
user_pdf['yelp_since_YRMO'] = user_pdf['yelping_since'].map(lambda x: 100*x.year + x.month)
user_pdf['yelp_since_year'] = user_pdf['yelping_since'].dt.year
 
user_pdf.head()
 

### Step 3: Convert back to spark and save

In [0]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
tipsDF=spark.createDataFrame(tips_pdf) 


